In [5]:
import numpy as np
condition_initial = np.array([
    [0.25],
    [0.75]
])
transicion = np.array([
    [0.7, 0.3],
    [0.1, 0.9]
])
emision = np.array([
    [0.9, 0.1],
    [0.2, 0.8]
])

class Viterbi(object):
    def __init__(self, condition_initial, transicion, emision):
        self.N = condition_initial.shape[0]
        self.initialProb = condition_initial.copy()
        self.transProb = transicion.copy()
        self.obsProb = emision.copy()
        
        #Validaciones en las dimensiones de datos de entrada
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N
 
    def Obs(self, obs):
        return self.obsProb[:, obs, None]
 
    def run(self, obs):
        trellis = np.zeros((self.N, len(obs)))
        backpt = np.ones((self.N, len(obs)), 'int32') * -1
 
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))
         
        for t in range(1, len(obs)):
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transProb).max(0)
            print("trellis: ", trellis[:, t])
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
        
        tokens = [trellis[:, -1].argmax()]
        for i in range(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]
    
algoritmo = Viterbi(condition_initial, transicion, emision)

In [6]:
print("#############TESTEO DE ALGORITMO #################")
print("\n Durante 5 días consecutivos se vio un paraguas")
print("Estados mas probables", algoritmo.run([1, 1, 1, 1, 1]))

print("\n Durante 5 días consecutivos NO se vio un paraguas")
print("Estados mas probables", algoritmo.run([0, 0, 0, 0, 0]))

print("\n Solo el Lunes Martes y Jueves se vio paraguas")
print("Estados mas probables", algoritmo.run([1, 1, 0, 1, 0]))

print("\n Solo Miercoles y Viernes se vio paraguas")
print("Estados mas probables", algoritmo.run([0, 0, 1, 0, 1]))

print("\n Solo el Jueves se vio paraguas")
print("Estados mas probables", algoritmo.run([0, 0, 0, 1, 0]))

#############TESTEO DE ALGORITMO #################

 Durante 5 días consecutivos se vio un paraguas
trellis:  [0.006 0.432]
trellis:  [0.00432 0.31104]
trellis:  [0.0031104 0.2239488]
trellis:  [0.00223949 0.16124314]
Estados mas probables [1, 1, 1, 1, 1]

 Durante 5 días consecutivos NO se vio un paraguas
trellis:  [0.14175 0.027  ]
trellis:  [0.0893025 0.008505 ]
trellis:  [0.05626057 0.00535815]
trellis:  [0.03544416 0.00337563]
Estados mas probables [0, 0, 0, 0, 0]

 Solo el Lunes Martes y Jueves se vio paraguas
trellis:  [0.006 0.432]
trellis:  [0.03888 0.07776]
trellis:  [0.0027216 0.0559872]
trellis:  [0.00503885 0.0100777 ]
Estados mas probables [1, 1, 1, 1, 1]

 Solo Miercoles y Viernes se vio paraguas
trellis:  [0.14175 0.027  ]
trellis:  [0.0099225 0.03402  ]
trellis:  [0.00625117 0.0061236 ]
trellis:  [0.00043758 0.00440899]
Estados mas probables [0, 0, 1, 1, 1]

 Solo el Jueves se vio paraguas
trellis:  [0.14175 0.027  ]
trellis:  [0.0893025 0.008505 ]
trellis:  [0.0062511